In [8]:
import pandas as pd
#import os
import csv
import country_converter as coco
import re
from pyproj import Transformer 


In [9]:

# Specify the filename
# File is in same directory as the code, so path of file doesn't need to be specified
filename = 'links.csv'
file_path = filename


print(file_path)


new_file = 'filtered_links.csv'
new_file_path = new_file


links.csv


## Create Filtered buses file

Create functions to get filtered file

In [10]:


def extract_coordinates(line_string):
    matches = re.findall(r'\(([^)]+)\)', line_string)
    if matches:
        # Split the first match by ',' to separate the source and destination coordinates
        src, dst = matches[0].split(',')
        
        # Split each coordinate by space to separate X and Y components and convert them to float
        src_x, src_y = tuple(map(float, src.strip().split(' ')))
        dst_x, dst_y = tuple(map(float, dst.strip().split(' ')))
        
        # Return the coordinates as tuples of floats
        return (src_x, src_y), (dst_x, dst_y)
    return (0.0, 0.0), (0.0, 0.0)  # Return zero coordinates if no matches are found

# Testing the function
src_coords, dst_coords = extract_coordinates('LINESTRING(22.939453 40.663973,23.192139 40.722283)')
print(f"Source Coordinates: {src_coords}")
print(f"Destination Coordinates: {dst_coords}")


def extract_tag_values(tags_string, tag_name):
    
    # tags are separated by a space
    tags_list = tags_string.split(" ")
    
    for tag in tags_list:
        tag = tag.strip("'")
        if "=>" in tag:
            tag_name_info, value = tag.split("=>")      #tag name and value separated by =>
        
            # goes through tags to see find specified tag name and info
            if tag_name_info.strip("'").strip() == tag_name:
                return value.strip("'").strip()         # additional .strip removes any extra spaces
        
    # Returns None if the specified tag name is not found
    return None



Source Coordinates: (22.939453, 40.663973)
Destination Coordinates: (23.192139, 40.722283)


In [11]:
with open(file_path, 'r', newline='') as csv_file, open(new_file_path, 'w', newline='') as new_csv_file:
    reader = csv.reader(csv_file)
    writer = csv.writer(new_csv_file)
    
    
    #Writing header to new file
    original_header = next(reader)
    geometry_index = original_header.index('geometry')
    new_header = original_header[:geometry_index] + ['src_coord', 'dst_coord', 'src_country', 'dst_country'] + original_header[geometry_index+1:]
    
    
    
    under_construction_index = new_header.index('under_construction')
    tags_index = new_header.index('tags')
    src_country_index = new_header.index('src_country')
    dst_country_index = new_header.index('dst_country')
    
    writer.writerow(new_header)
    
    
    for row in reader:
        under_construction_value = row[under_construction_index]
        
        # Rows where the buses are underconstruction are not included
        if under_construction_value != 't':
            
            
            ## Adjusting tags string
            # Combining the columns of tags in each row to be in one single column
            # Last two columns in rows contain details about coordinates, not tags
            last_tag_index = len(row) - 3
            
            tags_string = ' '.join(row[tags_index : last_tag_index])
            
            # manipulating the output of string
            tags_string = tags_string.replace('"', "'")
            if tags_string.startswith("''"):
                tags_string = tags_string[1:]
            if tags_string.endswith("''"):
                tags_string = tags_string[:-1]
                
            # Entire tags string is placed in single tags columns
            row[tags_index] = tags_string
            
            # Duplicate information is deleted
            row[tags_index + 1:-2] = []
            
            
            
            ## Adjusting the coordinates string
            # coordinates are split across two columns in a row
            combined_coords = f"{row[-2]},{row[-1]}"
            
            # Last two columns are replaced with the combined coordinates string
            row = row[:-2] + [combined_coords]
            
            
            # Extract source and destination coordinates from last columns in row
            src_coord, dst_coord = extract_coordinates(row[-1])
            
            # Last column in row that contained the string for coordinates is now replaced by 2 columns of source and destination coordinates
            row = row[:-1] + [src_coord, dst_coord]
            
            
            
            
            ## Extracting particular tag information
            
            # Access source and destination country and create new column
            src_country = extract_tag_values(tags_string, "country_1")
            row.append(src_country)
            
            dst_country = extract_tag_values(tags_string, "country_2")
            row.append(dst_country)
            
            if row[src_country_index] != 'NaN' or row[dst_country_index] != 'Nan':
                writer.writerow(row)
            
            #writer.writerow(row)

Check for any NaN values

In [12]:
df = pd.read_csv(new_file_path)
print(df.shape)
## 8889 14

nan_count1 = df['src_country'].isna().sum()
nan_count2 = df['dst_country'].isna().sum()

country_code_counts1 = df['src_country'].value_counts(dropna=False)
country_code_counts2 = df['dst_country'].value_counts(dropna=False)

print("\nNumber of NaN values in 'src_country' column:", nan_count1)
print("\nNumber of NaN values in 'dst_country' column:", nan_count2)

print("Number of occurrences of each country code1:")
print(country_code_counts1)
print("Number of occurrences of each country code2:")
print(country_code_counts2)

#nan_count 3


# Create a set of unique country codes from both 'src_country' and 'dst_country' columns excluding NaN
unique_country_codes = set(df['src_country'].dropna()).union(set(df['dst_country'].dropna()))

# Initialize a dictionary to store conversion results
conversion_results = {}

# Iterate over each unique country code
for code in unique_country_codes:
    # Convert the country code to the corresponding country name
    country_name = coco.convert(names=code, to='name_short')
    
    # Store the conversion result in the dictionary
    conversion_results[code] = country_name

# Print the conversion results
for code, name in conversion_results.items():
    print(f"{code}: {name}")

(8889, 14)

Number of NaN values in 'src_country' column: 1345

Number of NaN values in 'dst_country' column: 1407
Number of occurrences of each country code1:
src_country
NaN    1345
FR      905
RU      859
ES      689
DE      545
IT      369
GB      320
UA      229
SE      220
PL      204
EG      196
NO      188
DK      187
LY      185
MA      180
CH      176
TR      166
SY      151
DZ      149
RO      129
PT      119
TN      102
CZ       95
FI       92
BY       76
BG       72
AT       70
BA       67
JO       63
RS       60
BE       58
SK       54
HU       52
IE       52
IS       39
HR       36
NL       35
GR       33
LT       29
IL       28
AL       28
KZ       28
MD       26
SA       25
EE       24
LB       19
IQ       18
LV       17
CY       13
LU       11
ME       11
MK       11
SI       11
NI        9
GE        8
IR        3
AZ        2
AM        1
Name: count, dtype: int64
Number of occurrences of each country code2:
dst_country
NaN    1407
FR      910
RU      880
ES      670
D

# Create Country Specified dataframes and csv files

In [17]:
# Function to create country csv files


def get_country_network_data(file_path, country_id, voltages):
    # Load the DataFrame
    EU_df = pd.read_csv(file_path)

    # Filter based on country and voltage
    filtered_df = EU_df[((EU_df['src_country'] == country_id) | (EU_df['dst_country'] == country_id)) & EU_df['voltage'].isin(voltages)].copy()
    filtered_df.sort_values(by = 'voltage' , ascending = True, inplace = True)

    # Function to decide which bus ID to keep
    def decide_bus_id(existing_id, new_id):
        # Keeps the smaller bus ID
        return min(existing_id, new_id)

    # Dealing with duplicate nodes of the same coordinates but different bus IDs
    coord_to_bus_id = {}
    for index, row in filtered_df.iterrows():
        for coord_type in ['src_coord', 'dst_coord']:
            coord = row[coord_type]
            bus_id = row[coord_type.split('_')[0] + '_bus_id']
            
            if coord not in coord_to_bus_id:
                coord_to_bus_id[coord] = bus_id
            else:
                coord_to_bus_id[coord] = decide_bus_id(coord_to_bus_id[coord], bus_id)

    # Update the DataFrame using the mappings
    filtered_df.loc[:, 'src_bus_id'] = filtered_df['src_coord'].map(coord_to_bus_id)
    filtered_df.loc[:, 'dst_bus_id'] = filtered_df['dst_coord'].map(coord_to_bus_id)

    # Coordinate transformation
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)
    def transform_coord(coord):
        if isinstance(coord, str):
            coord = coord.strip("()").split(",")
            coord = tuple(map(float, coord))
        return transformer.transform(*coord)

    # Apply the coordinate transformation
    filtered_df[['src_x', 'src_y']] = filtered_df['src_coord'].apply(transform_coord).apply(pd.Series)
    filtered_df[['dst_x', 'dst_y']] = filtered_df['dst_coord'].apply(transform_coord).apply(pd.Series)

    # Save the filtered DataFrame to a CSV file named after the specified country
    output_file_path = f"{country_id}.csv"
    filtered_df.to_csv(output_file_path, index=False)
    print(f"File saved: {output_file_path}")

""" # Example usage of the function

new_file_path = 'path_to_your_data_file.csv'
process_electric_network_data(new_file_path, 'IE', [220, 380]) """


EU_filtered_data = new_file_path
get_country_network_data(EU_filtered_data, 'IE', [220, 380])

File saved: IE.csv


Ireland Dataframe for 220kV and 380kV

In [18]:
EU_filtered_data = new_file_path
get_country_network_data(EU_filtered_data, 'IE', [220, 380])

File saved: IE.csv
